In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/united-kingdom.csv')

C:\Users\james\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,5,41,81,90,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,RecordNo,endtime,qweek,i1_health,i2_health,i7a_health,i3_health,i4_health,i5_health_1,i5_health_2,...,WAH5,WAH7_1,WAH7_2,WAH7_3,WAH7_4,WAH7_5,WAH7_6,WAH7_7,WAH7_99,WAH6
0,1921,01/04/2020 17:42,week 1,3,2,1,"No, I have not","No, they have not",No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1922,01/04/2020 17:42,week 1,3,0,2,"No, I have not","No, they have not",No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1923,01/04/2020 17:46,week 1,1,12,1,"No, I have not","No, they have not",No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1924,01/04/2020 17:46,week 1,1,2,1,"No, I have not","No, they have not",No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1925,01/04/2020 17:46,week 1,2,0,1,"No, I have not","No, they have not",No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info(verbose=True, max_cols=470)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52391 entries, 0 to 52390
Data columns (total 461 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RecordNo            52391 non-null  int64  
 1   endtime             52391 non-null  object 
 2   qweek               52391 non-null  object 
 3   i1_health           52391 non-null  object 
 4   i2_health           52391 non-null  int64  
 5   i7a_health          52391 non-null  object 
 6   i3_health           52391 non-null  object 
 7   i4_health           52391 non-null  object 
 8   i5_health_1         52391 non-null  object 
 9   i5_health_2         52391 non-null  object 
 10  i5_health_3         52391 non-null  object 
 11  i5_health_4         52391 non-null  object 
 12  i5_health_5         52391 non-null  object 
 13  i5_health_99        52391 non-null  object 
 14  i5a_health          52391 non-null  object 
 15  i6_health           52391 non-null  object 
 16  i7b

In [6]:
df_q = pd.read_excel('codebook.xlsx')

In [7]:
df_q.drop(df_q.index[:50], inplace=True)

In [8]:
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
50,NaN,35,week 35
51,NaN,36,week 36
52,NaN,37,week 37
53,NaN,38,week 38
54,NaN,39,week 39
55,NaN,40,week 40
56,NaN,41,week 41
57,NaN,42,week 42
58,NaN,43,week 43
59,NaN,44,week 44


In [9]:
df_q.dropna(axis=0, subset=['RecordNo'], inplace=True) 

In [10]:
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
68,i1_health,NaN,NaN
70,Standard Attributes,Label,About how many people from your household have...
74,i2_health,NaN,NaN
76,Standard Attributes,Label,"Not including those people in your household, ..."
80,i7a_health,NaN,NaN
82,Standard Attributes,Label,How many times did you leave your home yesterd...
86,i3_health,NaN,NaN
88,Standard Attributes,Label,"In the last 7 days, have you personally been t..."
91,Valid Values,1,"Yes, and I tested positive"
96,i4_health,NaN,NaN


In [11]:
df_q.reset_index(drop=True, inplace=True)

In [12]:
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
0,i1_health,NaN,NaN
1,Standard Attributes,Label,About how many people from your household have...
2,i2_health,NaN,NaN
3,Standard Attributes,Label,"Not including those people in your household, ..."
4,i7a_health,NaN,NaN
5,Standard Attributes,Label,How many times did you leave your home yesterd...
6,i3_health,NaN,NaN
7,Standard Attributes,Label,"In the last 7 days, have you personally been t..."
8,Valid Values,1,"Yes, and I tested positive"
9,i4_health,NaN,NaN


In [13]:
for index, row in df_q.iterrows():
    if pd.isna(row['Unnamed: 2']) == True:
        df_q.iloc[index]['Unnamed: 2'] = df_q.iloc[index+1]['Unnamed: 2']
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
0,i1_health,NaN,About how many people from your household have...
1,Standard Attributes,Label,About how many people from your household have...
2,i2_health,NaN,"Not including those people in your household, ..."
3,Standard Attributes,Label,"Not including those people in your household, ..."
4,i7a_health,NaN,How many times did you leave your home yesterd...
5,Standard Attributes,Label,How many times did you leave your home yesterd...
6,i3_health,NaN,"In the last 7 days, have you personally been t..."
7,Standard Attributes,Label,"In the last 7 days, have you personally been t..."
8,Valid Values,1,"Yes, and I tested positive"
9,i4_health,NaN,"In the last 7 days, has someone in your househ..."


In [14]:
df_q=df_q[df_q['RecordNo'] != 'Standard Attributes']

In [15]:
df_q=df_q[df_q['RecordNo'] != 'Valid Values']

In [16]:
df_q.drop('Unnamed: 1', axis=1, inplace=True)

In [17]:
df_q.reset_index(drop=True, inplace=True)

In [18]:
df_q.head(50)

,RecordNo,Unnamed: 2
0,i1_health,About how many people from your household have...
1,i2_health,"Not including those people in your household, ..."
2,i7a_health,How many times did you leave your home yesterd...
3,i3_health,"In the last 7 days, have you personally been t..."
4,i4_health,"In the last 7 days, has someone in your househ..."
5,i5_health_1,Dry cough
6,i5_health_2,Fever
7,i5_health_3,Loss of sense of smell
8,i5_health_4,Loss of sense of taste
9,i5_health_5,Shortness of breath or difficulty breathing


In [19]:
df_q.rename(columns = {'RecordNo':'code', 'Unnamed: 2': 'question'}, inplace = True)
df_q

,code,question
0,i1_health,About how many people from your household have...
1,i2_health,"Not including those people in your household, ..."
2,i7a_health,How many times did you leave your home yesterd...
3,i3_health,"In the last 7 days, have you personally been t..."
4,i4_health,"In the last 7 days, has someone in your househ..."
...,...,...
505,Multiple Response Categories,I'm not eligible to receive one yet
506,$child_age,"In which, if any, any of the following age gro..."
507,Multiple Response Categories,0 - 2
508,$WAH7,If you are likely to further away from the cen...


In [20]:
for index, row in df_q.iterrows():
    print(row['question'])

About how many people from your household have you come into physical contact with (within 2 meters / 6 feet)?
Not including those people in your household, about how many people have you come into physical contact with (within 2 meters / 6 feet)?
How many times did you leave your home yesterday?  If you are not staying at home, how many times did you leave where you are staying yesterday?
In the last 7 days, have you personally been tested for coronavirus (COVID-19?)
In the last 7 days, has someone in your household been tested for coronavirus (COVID-19)?
Dry cough
Fever
Loss of sense of smell
Loss of sense of taste
Shortness of breath or difficulty breathing
None of these
Thinking about the 7 days before your symptoms started, had you been in physical contact (within 2 meters / 6 feet) with someone who has a confirmed diagnosis of coronavirus (Covid-19), or someone with the following symptoms: dry cough, fever, loss of sen
Since developing those symptoms, to what extent have you self